<a href="https://colab.research.google.com/github/shreekrishh/Data-Analytics/blob/main/Genre%20detection%20through%20the%20plot%20of%20book%20using%20NLP/Genre_detection_through_the_plot_of_book_using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sklearn



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# load the training data

train_data = json.load(open("/content/drive/MyDrive/ak-as2-15-1/4650/comp4650-A2/Q2/genre_train.json", "r"))
# train_data = json.load(open("/content/drive/MyDrive/comp4650-A2/Q2/genre_train.json", "r"))


In [ ]:
X = train_data['X']
Y = train_data['Y'] # id mapping: 0 - Horror, 1 - Science Fiction, 2 - Humor, 3 - Crime Fiction
docid = train_data['docid'] # these are the ids of the books which each training example came from

# load the test data
# the test data does not have labels, our model needs to generate these
test_data = json.load(open("/content/drive/MyDrive/ak-as2-15-1/4650/comp4650-A2/Q2/genre_test.json", "r"))
# test_data = json.load(open("/content/drive/MyDrive/comp4650-A2/Q2/genre_test.json", "r"))

Xt = test_data['X']


In [ ]:
df = pd.DataFrame(list(zip(X, Y,docid)),
               columns =['plot', 'genre','Book ID'])
df

,plot,genre,Book ID
0,"""Go right in there."" He gaped at my arm when t...",1,1212
1,"""He'll have it. In writing, if he wishes."" ""Ye...",1,580
2,"""What we call Unit Eight--the heart of the dri...",1,259
3,"""Excuse me,"" he said, ""I think--""\n\n""Thank yo...",2,869
4,"""Well, moderately so, Barncastle, moderately s...",2,399
...,...,...,...
6738,"""They made it,"" he muttered. ""Both of them. Go...",1,938
6739,"Perhaps he had come to murder me, as you surmi...",3,1085
6740,Bruce listened to the wind. It seemed to rise ...,1,440
6741,Swift fleeting batlike shapes would appear\nfr...,1,261


In [ ]:
df_y = pd.DataFrame(list(zip(Xt)),
               columns =['plot'])
df_y

,plot
0,"It was too awful. I never thought, never for a..."
1,A cold\nwind blew across the road leading up t...
2,"""That is the Archangel\nMichael overcoming Sat..."
3,Longer plumes of changeable\ncolours radiated ...
4,"""What it should be. It checks."" Tom whooped an..."
...,...
47178,"""Roger, it'd take a man two weeks to go from o..."
47179,"""How is he--awake?"" ""About the same--still unc..."
47180,"""Yes, brother,"" Volna said softly. ""They do. O..."
47181,"she snapped as her breath came back. ""Beat it!..."


In [ ]:
def cleaning(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    
    return text

In [ ]:
len(df['Book ID'].unique())

758

In [ ]:
df['clean_plot'] = df['plot'].apply(lambda x: cleaning(x))
df



,plot,genre,Book ID,clean_plot
0,"""Go right in there."" He gaped at my arm when t...",1,1212,go right in there he gaped at my arm when the ...
1,"""He'll have it. In writing, if he wishes."" ""Ye...",1,580,hell have it in writing if he wishes yes i ass...
2,"""What we call Unit Eight--the heart of the dri...",1,259,what we call unit eight the heart of the drive...
3,"""Excuse me,"" he said, ""I think--""\n\n""Thank yo...",2,869,excuse me he said i think thank you said the g...
4,"""Well, moderately so, Barncastle, moderately s...",2,399,well moderately so barncastle moderately so fa...
...,...,...,...,...
6738,"""They made it,"" he muttered. ""Both of them. Go...",1,938,they made it he muttered both of them good he ...
6739,"Perhaps he had come to murder me, as you surmi...",3,1085,perhaps he had come to murder me as you surmis...
6740,Bruce listened to the wind. It seemed to rise ...,1,440,bruce listened to the wind it seemed to rise h...
6741,Swift fleeting batlike shapes would appear\nfr...,1,261,swift fleeting batlike shapes would appear fro...


In [ ]:
df['genre'].value_counts()


1    4695
2    1328
3     386
0     334
Name: genre, dtype: int64

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))
def no_stopword(text):
  no_stopword_list=[]
  for word in text.split():
    if word not in stop_words:
      no_stopword_list.append(word)
  return ' '.join(no_stopword_list)
df['clean_plot'] = df['clean_plot'].apply(lambda x: no_stopword(x))






In [ ]:

feature = df['genre']

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5)

In [ ]:
X = df['clean_plot']
y = feature
scores = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    tfidf= TfidfVectorizer(max_df=0.50, max_features=5000)
    xtrain_tfidf = tfidf.fit_transform(X_train)
    xval_tfidf = tfidf.transform(X_test)
    logisreg = LogisticRegression(class_weight='balanced',C=0.1)
    # logisreg = GaussianNB()
    #logisreg = MultinomialNB()
    #logisreg = DecisionTreeClassifier()
    logisreg.fit(xtrain_tfidf.toarray(),y_train)
    ypredict=logisreg.predict(xval_tfidf.toarray())
    macro_score=f1_score(y_test, ypredict, average="macro")
    scores.append(macro_score)
print(scores)
print(np.average(scores))

[0.6994124521985624, 0.6991803957153165, 0.7271315322073606, 0.7168326701327998, 0.7017573092142686]
0.7088628718936616


In [ ]:
xtrain, xval, ytrain, yval = train_test_split(df['clean_plot'],feature, test_size=0.2, random_state=42,stratify=feature)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf= TfidfVectorizer(max_df=0.5, max_features=5000)
xtrain_tfidf = tfidf.fit_transform(xtrain)
xval_tfidf = tfidf.transform(xval)
print(xtrain_tfidf)



  (0, 4496)	0.037521308655100186
  (0, 608)	0.06637052958709648
  (0, 4607)	0.10369307620074732
  (0, 4790)	0.08202492666097595
  (0, 3925)	0.07150647805576649
  (0, 3806)	0.06119577309935585
  (0, 4557)	0.09032662839457829
  (0, 2210)	0.09949649614985864
  (0, 4000)	0.10369307620074732
  (0, 4210)	0.06684885315852516
  (0, 489)	0.08377753925169851
  (0, 3293)	0.08096354507367402
  (0, 4974)	0.07927384129009044
  (0, 2177)	0.06139515062588238
  (0, 4624)	0.04103927509646983
  (0, 4269)	0.09949649614985864
  (0, 3816)	0.03983089973423022
  (0, 1112)	0.07882495781329575
  (0, 2821)	0.051217060782257615
  (0, 1986)	0.1502687109759507
  (0, 1320)	0.0861300483436896
  (0, 2060)	0.05440572290263885
  (0, 2474)	0.06042444479123771
  (0, 2281)	0.07150647805576649
  (0, 86)	0.06927692669722466
  :	:
  (5393, 2867)	0.06263775927009972
  (5393, 3835)	0.06658549855684776
  (5393, 164)	0.046673845535678646
  (5393, 4808)	0.03857333737500646
  (5393, 4602)	0.05786934928316631
  (5393, 2746)	0.063748

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

logisreg = LogisticRegression(class_weight='balanced', C=0.1)
# logisreg = GaussianNB()
#logisreg = MultinomialNB()
# logisreg = DecisionTreeClassifier()
# logisreg = SVC()
logisreg.fit(xtrain_tfidf.toarray(),ytrain)
yp=logisreg.predict(xtrain_tfidf.toarray())
print(f1_score(ytrain, yp, average="macro"))
ypredict=logisreg.predict(xval_tfidf.toarray())
macro_score=f1_score(yval, ypredict, average="macro")
macro_score


0.8454783487804585


0.7129307682243015

In [ ]:
Xt_tfidf = tfidf.transform(Xt)
model = logisreg

In [ ]:
# predict on the test data
Y_test_pred = model.predict(Xt_tfidf)


fout = open("out.csv", "w")
fout.write("Id,Y\n")
for i, line in enumerate(Y_test_pred):
    fout.write("%d,%d\n" % (i, line))
fout.close()

In [ ]:
final_dict= {'plot':test_data['X'],
             'genre': Y_test_pred}
final_df = pd.DataFrame(final_dict)
final_df['genre'].value_counts()
final_df.to_csv('C:\KDAG material\plot-genre.csv')
